<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/bond_analysis_report_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
url = 'https://finance.naver.com/research/debenture_list.nhn?'
params = {'page': 126}

resp = requests.get(url,params=params)
soup = BeautifulSoup(resp.content, 'html.parser')
  
td_tag = soup.findAll("td",{"class":"file"})
date_tag = soup.findAll('td', {"style":"padding-left:5px"} )

# print(td_tag)

pdf_list_ = []
date_list_ = []

for t in td_tag:
  pdf_url=t.find('a').attrs['href']
  pdf_list_.append(pdf_url)



for date in date_tag:
  date_list_.append(date.get_text())

date_list_, pdf_list_



(['08.05.19',
  '08.05.13',
  '08.05.04',
  '08.04.28',
  '08.04.21',
  '08.04.14',
  '08.04.07',
  '08.04.07',
  '08.04.01'],
 ['http://imgstock.naver.com/upload/research/debenture/1211247153.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1210647721.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1210033641.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1209352859.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1208928986.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1208929274.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1213152088.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1208929417.pdf',
  'http://imgstock.naver.com/upload/research/debenture/1208930733.pdf'])

In [20]:
for pdf,date in zip(pdf_list_, date_list_):
  with open('/content/drive/My Drive/bond_analysis_report/bond_analysis_report_{}.pdf'.format(date), "wb") as file:
    response = requests.get(pdf)
    file.write(response.content)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import requests
from bs4 import BeautifulSoup

pdf_list = []
date_list = []

for i in range(23,127):
  url = 'https://finance.naver.com/research/debenture_list.nhn?'
  params = {
      'page' : i
  }

  resp = requests.get(url,params=params)
  soup = BeautifulSoup(resp.content, 'html.parser')

  td_tag = soup.findAll("td",{"class":"file"})
  date_tag = soup.findAll('td', {"style":"padding-left:5px"} )


  for t in td_tag:
    pdf_url=t.find('a').attrs['href']
    pdf_list.append(pdf_url)



  for date in date_tag:
    date_list.append(date.get_text())


In [32]:
print(len(pdf_list))   
print(len(date_list))

print(date_list[-10:])
print(date_list[:10])

3099
3099
['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['18.01.02', '18.01.02', '17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14']


In [34]:
pdf_list_ = pdf_list[2:]
date_list_ = date_list[2:]

print(date_list_[-10:])
print(date_list_[:10])

['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14', '17.12.14', '17.12.13']


In [35]:
print(len(pdf_list_))   
print(len(date_list_))


3097
3097


In [ ]:

for pdf,date in zip(pdf_list, date_list):
  with open('/content/drive/My Drive/bond_analysis_report/bond_analysis_report_{}.pdf'.format(date), "wb") as file:
    response = requests.get(pdf)
    file.write(response.content)

In [ ]:
!pip install tika
from tika import parser
rawText = parser.from_file('/content/drive/My Drive/bond_analysis_report/채권_분석_리포트_08.06.26.pdf')
rawList = rawText['content'].splitlines()

In [ ]:
rawText = parser.from_file('/content/drive/My Drive/bond_analysis_report/채권_분석_리포트_08.06.26.pdf')
rawList = rawText['content']

rawList

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMicrosoft Word - 08년 Daewoo Bond Brief.doc\n\n\nDaewoo Daewoo Bond Brief   \n채권   윤일광 (02-768-3058) yuniil1@bestez.com \n\n \n\n \n\n \n\n2008. 6. 26. Thu \n\n채권시황 \n\n 전일 3년 만기와 10년 만기 금리는 각각 3bp씩 하락한 5.82%, 5.97%로 마감 \n\n 장 초반 금리는 향후 정책 불확실성으로 관망세를 보이며 보합권에서 거래. 이후 국채선물 저평가가 30틱 수준을 유지하고 저가 메리트\n가 부각되자 매수세 늘며 다소 금리 하락. 다만, 원/달러 환율 상승과 FOMC 결과를 지켜보자는 심리에서 금리 낙폭은 제한되었음. \n\n 전일 FOMC 회의에서 서프라이즈한 결과는 없었음. 성명서 내용은 버냉키 의장 및 여타 연준 인사들이 그 동안 언급해온 내용과 크게 \n다르지 않아 시장에 큰 영향을 미치지 못함. 연준은 통화정책의 스탠스가 바뀔 수 있음을 시사했는데, 뚜렷하게 금리인상 가능성을 포함\n\n하지 않았다는 점에서 시장이 민감하게 반응할 가능성을 견제. 이미 가늠할 수 있는 범위에서 FOMC 회의가 끝났기 때문에 시장에 미칠 \n\n영향은 미미하겠지만, 물가 우려 발언에도 불구하고 조만간 인상 가능성이 낮아져 미 금리가 다소 하락한 것은 국내 시장 참가자들의 심\n\n리 개선에 도움을 줄 것으로 판단됨. \n\n \n\n관심 차트 및 경제지표 \n \n\n전일에 대한 기술적 반락 이어지며 금리 하락  FOMC에서 물가 우려 강조했지만, 조만간 인상 가능성은 낮아 하락.\n\n   \n\n자료: 증권업협회, 대우증권리서치 EZ-DB  자료: Thomson Datastream, 대우증권리서치 EZ-DB \n\n \n\n소비 관련 지수 전부 악화 일로  유가 수요

In [ ]:
import os

path = "/content/drive/My Drive/bond_analysis_report"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

len(file_list)